<a href="https://colab.research.google.com/github/KinoshitaYstr/mmi-learning/blob/master/chap04a.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## chap04

1.   データが置かれたGoogleドライブをマウント（Colabのメニューで操作）


> ファイル -> ドライブをマウント -> 実行可能なセルが現れる -> ログインして認証コードを入力




2.   /content/driveにマウントされる



In [1]:
from google.colab import drive
drive.mount('/content/drive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/drive


GiNZAのインストール  

In [2]:
!pip install -U ginza
import pkg_resources, imp
imp.reload(pkg_resources)

     |████████████████████████████████| 54.9MB 55kB/s 
     |████████████████████████████████| 81kB 12.5MB/s 
     |████████████████████████████████| 44.8MB 69kB/s 
     |████████████████████████████████| 481kB 49.4MB/s 
  Created wheel for ginza: filename=ginza-3.1.2-cp36-none-any.whl size=17311 sha256=41668821f137c8391a7056cc0782d4c34fffd645e94d2e87258b63416b547cc1
  Stored in directory: /root/.cache/pip/wheels/70/8d/57/f089078acc0dbaebffc08c178e9f20924fa794c114ad36f7f7
  Created wheel for ja-ginza: filename=ja_ginza-3.1.0-cp36-none-any.whl size=54963619 sha256=4ec07c4dc44d715471667d5a2fe1772a29026e8d13f9026ae4a9d1d262cb7bed
  Stored in directory: /root/.cache/pip/wheels/f2/8a/07/1837eeb5c5648fa8d266102b78a894e495234585ac3f024cf1
  Created wheel for ja-ginza-dict: filename=ja_ginza_dict-3.1.0-cp36-none-any.whl size=70877544 sha256=0b87a54a0716aa5e4948dc9ecc0d0f45716a56aeb5d4056b8627949c45ab2e08
  Stored in directory: /root/.cache/pip/wheels/30/88/d7/7f0692ba26060966af34538e1079438d16

<module 'pkg_resources' from '/usr/local/lib/python3.6/dist-packages/pkg_resources/__init__.py'>

GiNZAの使用準備

In [0]:
import spacy
nlp = spacy.load('ja_ginza')

京都大学情報学研究科--NTTコミュニケーション科学基礎研究所 共同研究ユニット 
 [解析済みブログコーパス](http://nlp.ist.i.kyoto-u.ac.jp/kuntt/#ga739fe2)
の読み込み

In [0]:
Dpath = '/content/drive/My Drive/Resources/KNBC_v1.0_090925_utf8/corpus2/'
topics = ['Gourmet', 'Keitai', 'Kyoto', 'Sports']

In [0]:
import pandas as pd
import re

cols = ['doc', 'label']
df = pd.DataFrame(index=[], columns=cols)
p = re.compile(r'(［.*］)(.*)')

for topic in topics:
  data = pd.read_table(Dpath+topic+'.tsv', header=None, dtype=str, usecols=[1])
  for d in data.itertuples():
    m = p.match(d[1])
    if m != None:  # first line of the document
      if d[0] != 0:
        df = df.append(pd.DataFrame([doc.strip(), topic], index = cols).T)
      sent = m.groups()[1]
      doc = ''
    else:
      sent = d[1]
    doc = doc + ' ' +  ' '.join([t.text for t in nlp(sent)])
  df = df.append(pd.DataFrame([doc.strip(), topic], index = cols).T)

In [62]:
df

,doc,label
0,烏丸 六角 の おかき 屋 さん 六角堂 の 前 に ある 、 蕪村 庵 と いう お店 に...,Gourmet
0,河原町 の 居酒屋 この 間 先輩 たち に つれ られ 、 河原町 の 居酒屋 へ 行っ ...,Gourmet
0,ちょっと 贅沢 ほんの ちょこっと な ん だ けど 、 贅沢 し たい とき に 何 を ...,Gourmet
0,食欲 の 秋 最近 おなか が 減っ て 仕方 あり ませ ん か 。 仕方 あり ませ ん...,Gourmet
0,季節 限定 も のっ て … １０ 月 と いえ ば 秋 本番 で ある 。 栗 や 秋刀魚...,Gourmet
...,...,...
0,クライマックス シリーズ 今年 の 阪神 は なん やっ た ん や ！ ！ 初め 弱く て...,Sports
0,京都 サンガ に つい て この ブログ の 右側 に 検索 窓 ある じゃ ない です か...,Sports
0,サッカー から 伺える ルール と の 付き合い 方 私自身 は 全く の サッカー 素人 ...,Sports
0,する か 見る か スポーツ は する 派 です か ？ し たい 見る 派 です か ？ ...,Sports


In [63]:
df['label'].value_counts(sort=False)

Keitai     79
Sports     22
Gourmet    57
Kyoto      91
Name: label, dtype: int64

# 文書分類

test

方針
* 一回すべての文からBoWを作成or取得する
* ラベルの用意
* それらをscikit-learnのモデルに入れる？

In [195]:
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import CountVectorizer, TfidfTransformer

# ラベルの生成
labels = df['label'].unique()
print(labels)
# 訓練・テスト分け
train_df, test_df = train_test_split(df, test_size=0.3, shuffle=True)
print(train_df.shape, test_df.shape)

# 訓練データからBoWとベクトル化例
# sklearn kitのCountVectorizerを使う
count = CountVectorizer()
# fitで学習
count.fit(train_df['doc'])
# transformでBoWに変換する
train_bow = count.transform(train_df['doc'])

# ttf-tfidfでBoWをベクトル化例
ttf = TfidfTransformer()
ttf.fit(train_bow)
train_tfidf = ttf.transform(train_bow)

# テストデータもBoWとベクトル化例
test_bow = count.transform(test_df['doc'])
test_tfidf = ttf.transform(test_bow)


['Gourmet' 'Keitai' 'Kyoto' 'Sports']
(174, 2) (75, 2)


In [0]:
# 訓練データの前処理
x_train = train_df.doc
t_train = train_df.label

bow_train = count.transform(x_train)
tfidf_train = ttf.transform(bow_train)

# テストデータの前処理
x_test = test_df.doc
t_test = test_df.label

bow_test = count.transform(x_test)
tfidf_test = ttf.transform(bow_test)

## 単純ベイズ

学習

In [197]:
from sklearn.naive_bayes import MultinomialNB

clf_bayes = MultinomialNB(alpha=0.02)
clf_bayes.fit(tfidf_train, t_train)

MultinomialNB(alpha=0.02, class_prior=None, fit_prior=True)

テスト

In [198]:
y_test = clf_bayes.predict(tfidf_test)

tmp = [t == y for t, y in zip(t_test, y_test) ]
accuracy = 100*(sum(tmp)/len(tmp))
print(str(accuracy)+"%")

90.66666666666666%


## ロジスティック回帰

学習

In [200]:
from sklearn.linear_model import LogisticRegression

clf_rogistic = LogisticRegression()
clf_rogistic.fit(tfidf_train, t_train)

LogisticRegression(C=1.0, class_weight=None, dual=False, fit_intercept=True,
                   intercept_scaling=1, l1_ratio=None, max_iter=100,
                   multi_class='auto', n_jobs=None, penalty='l2',
                   random_state=None, solver='lbfgs', tol=0.0001, verbose=0,
                   warm_start=False)

テスト

In [201]:
y_test = clf_rogistic.predict(tfidf_test)

tmp = [t == y for t, y in zip(t_test, y_test) ]
accuracy = 100*(sum(tmp)/len(tmp))
print(str(accuracy)+"%")

78.66666666666666%


## オンラインSVM

学習

In [245]:
from sklearn.svm import LinearSVC

clf_svm = LinearSVC()
clf_svm.fit(tfidf_train, t_train)

LinearSVC(C=1.0, class_weight=None, dual=True, fit_intercept=True,
          intercept_scaling=1, loss='squared_hinge', max_iter=1000,
          multi_class='ovr', penalty='l2', random_state=None, tol=0.0001,
          verbose=0)

テスト

In [246]:
y_test = clf_svm.predict(tfidf_test)

tmp = [t == y for t, y in zip(t_test, y_test) ]
accuracy = 100*(sum(tmp)/len(tmp))
print(str(accuracy)+"%")

85.33333333333334%
